<a href="https://colab.research.google.com/github/A01720548/M1-Actividad-Multiagentes/blob/main/M1_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esteban de la Maza A01720548

In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 34.5 MB/s 
     |████████████████████████████████| 60 kB 8.3 MB/s 


In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `Multigrid` que nos deja tener varios agentes en una celda
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import signal

In [ ]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
          #if agent.unique_id[2] == 1:
            grid[x][y] = agent.status
    return grid

class VaccuumAgent(Agent):
    def __init__(self, unique_id, model):
      super().__init__(unique_id, model)
      self.status = 3

    def random_move(self):
      '''
      Este metodo nos entrega de manera aleatoria una lista de posiciones donde se pueda mover 
      de una cuadricula a la vez y desplaza a los objetos de tipo Vaccuum 
      '''
      next_moves = self.model.grid.get_neighborhood(self.pos, True, True)
      next_move = self.random.choice(next_moves)
      # Realizar el movimiento
      self.model.grid.move_agent(self, next_move)

    def step(self):
      '''
      Este método es el que calcula si esta limpio o sucio el piso de la posicion donde se encuentra actualmente
      a traves de su unique ID y una verificacion de que tipo de objeto es. En caso de que se tenga que limpiar, 
      se actualiza el estado del piso 
      '''
      neighbours = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=True)
      
      for neighbour in neighbours:
        if (neighbour.unique_id[0] == self.pos[0] and neighbour.unique_id[1] == self.pos[1] and neighbour.unique_id[2] == 1):
          neighbour.status = 0
        
    def advance(self):
      '''
      Define el nuevo estado despues de llamar a la funcion para moverse de manera aleatoria
      '''
      self.random_move()


class FloorAgent(Agent):
    '''
    Representa a un agente o una celda con estado sucio (1) o limpio (0)
    '''
    def __init__(self, unique_id, model, isDirty):
      '''
      Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
      formado por una tupla (x,y,1). 
      '''
      super().__init__(unique_id, model)
      self.status = isDirty
       

class VaccuumAgentModel(Model):
    '''
    Define el modelo donde se encuentran los agentes de aspiradoras y piso. Se crea el MultiGrid y se agregan agentes
    a las casillas. Si es tipo vaccuum su posicion inicial es (1,1) y se agrega un 0 para identificarlo como Vaccuum
    Si es tipo Floor se agrega a cada casilla y se le da un 1 en su Unique ID para identificarlo
    '''
    def __init__(self, width, height, numAgents, maxRunTime, percentageDirty):
      self.num_agents = width * height
      self.grid = MultiGrid(width, height, True)
      self.schedule = SimultaneousActivation(self)
      self.maxTime = datetime.timedelta(seconds=maxRunTime)
      


      # Percentage of Initial Dirty Floors
      gridSize = width*height
      total = int(gridSize * percentageDirty/100)
      print(total)
      arrayOfOnes = np.zeros((width,height), int)
      x = 0
      while x < total:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if arrayOfOnes[pos][pos2] == 0:
          arrayOfOnes[pos][pos2] = 1
          x+=1
      print(arrayOfOnes)
      for (content, x, y) in self.grid.coord_iter():
        if numAgents > 0:
          a = VaccuumAgent((x, y, 0), self)
          self.grid.place_agent(a, (1,1)) # Inicializar los Agentes en la celda 1,1
          self.schedule.add(a)
          numAgents-=1
        b = FloorAgent((x,y, 1), self, arrayOfOnes[x][y])
        self.grid.place_agent(b, (x,y))
        self.schedule.add(b)

      
      # Aquí definimos con colector para obtener el grid completo.
      self.datacollector = DataCollector(
          model_reporters={"Grid": get_grid})
    
    def step(self):
      '''
      En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
      '''
      self.datacollector.collect(self)
      self.schedule.step()

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 100

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 50

# Registramos el tiempo de inicio y corremos el modelo

# Max RunTime
start_time = time.time()
elapsed = datetime.timedelta(seconds=(time.time() - start_time))
maxTime = datetime.timedelta(seconds=5)

model = VaccuumAgentModel(GRID_SIZE, GRID_SIZE, 100, 3, 50)
for i in range(NUM_GENERATIONS):
  if elapsed < maxTime:
    model.step()
    elapsed = datetime.timedelta(seconds=(time.time() - start_time))
    print(elapsed)


# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

5000
[[1 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 0]
 ...
 [1 0 1 ... 0 1 0]
 [1 1 1 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
0:00:00.119589
0:00:00.133912
0:00:00.148294
0:00:00.162791
0:00:00.176737
0:00:00.190760
0:00:00.212019
0:00:00.229609
0:00:00.246609
0:00:00.260466
0:00:00.277625
0:00:00.293532
0:00:00.307376
0:00:00.321158
0:00:00.336029
0:00:00.350652
0:00:00.364646
0:00:00.380723
0:00:00.394749
0:00:00.414372
0:00:00.434478
0:00:00.449329
0:00:00.464397
0:00:00.480742
0:00:00.495781
0:00:00.512031
0:00:00.527648
0:00:00.543803
0:00:00.558009
0:00:00.572023
0:00:00.586792
0:00:00.600692
0:00:00.614681
0:00:00.628679
0:00:00.730634
0:00:00.749601
0:00:00.766220
0:00:00.780755
0:00:00.796165
0:00:00.809841
0:00:00.823514
0:00:00.837228
0:00:00.853647
0:00:00.867596
0:00:00.881382
0:00:00.895175
0:00:00.908946
0:00:00.922942
0:00:00.937069
0:00:00.955910
Tiempo de ejecución: 0:00:00.956636


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Blues)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim


# Conclusiones

Probe de 2 agentes a 50 agentes y me di cuenta que el tiempo se afecta de la siguiente manera
